<a href="https://colab.research.google.com/github/Sastrybbch/git-example/blob/main/Llama3_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-storage 2.25.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigtable 2.24.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but 

In [ ]:
!pip install llama-parse

# Installation of packages

In [4]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

In [5]:
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [6]:
def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [ ]:
!mkdir data

In [ ]:
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

In [ ]:
Quarter1 =

##Document Parsing

In [12]:
instruction = """The provided document is LTIMindtree First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key=userdata.get("LLAMA_PARSE"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("/bin/LTIMindtreeQ1.pdf")

Started parsing the file under job_id 083caa95-ffab-4bbb-b67b-e12f4f989830


In [13]:
parsed_doc = llama_parse_documents[0]

In [17]:
Markdown(parsed_doc.text[:4096])

# LTIMindtree

# Earnings Release & Fact Sheet

# Fourth Quarter, Fiscal 2024

April 24, 2024
---
# Contents

Press Release ................................................................. 3

Key Metrics ................................................................. 8

Financial Statements ....................................................... 11

Full Year Performance ..................................................... 14

Contact Information ......................................................... 19

Safe Harbour

Certain statements in this release concerning the future prospects are forward-looking statements. These statements by their nature involve risks and uncertainties that could cause the actual results to differ materially from such forward-looking statements. The Company assumes no obligation to revise or update any forward-looking statements that may be made from time to time by or on behalf of the Company.

Deal wins and recognitions section in this release include relevant disclosures between our last earnings release and this release.
---
# LTIMindtree Closes FY24 with Strong Order Inflow

Revenue growth of 4.4% in USD; Growth broad-based across all verticals

Mumbai, April 24, 2024: LTIMindtree [NSE: LTIM, BSE: 540005], a global technology consulting and digital solutions company, announced its consolidated results today for the fourth quarter and full year ended March 31, 2024, as approved by its Board of directors.

“We closed FY24 amidst a tough macro environment and delivered a resilient performance with full-year revenue growth of 4.4% in USD terms and an EBIT margin of 15.7%. Our order inflow for the full year at USD 5.6 billion registered a 15.7% growth over FY23. This growth reflects the positive outcomes of our positioning as an organization with scale, expanded capabilities, and larger partnerships. As the market dynamics evolve, we are excited to be part of innovations, partnerships, and initiatives that our clients will embark on in FY 25.”

-Debashis Chatterjee, Chief Executive Officer and Managing Director

# Key financial highlights:

# Quarter ended March 31, 2024

| |In USD|In INR|
|---|---|---|
|Revenue|$1,069.4 million (-1.3% Q-o-Q / +1.1% Y-o-Y)|₹88,929 million (-1.4% Q-o-Q / +2.3% Y-o-Y)|
|Net profit|$132.4 million (-5.8% Q-o-Q / -2.4% Y-o-Y)|₹11,007 million (-5.9% Q-o-Q / -1.2% Y-o-Y)|

# Year ended March 31, 2024

| |In USD|In INR|
|---|---|---|
|Revenue|$4,287.3 million (growth of 4.4% Y-o-Y)|₹3,55,170 million (growth of 7.0% Y-o-Y)|
|Net profit|$553.4 million (growth of 1.4% Y-o-Y)|₹45,846 million (growth of 4.0% Y-o-Y)|
---
# Other highlights:

|Clients:|738 active clients as of March 31, 2024|
|---|---|
|$5 million+ clients increased by 7 on a Y-o-Y basis, total 153| |
|$10 million+ clients increased by 10 on a Y-o-Y basis, total 91| |
|$20 million+ clients increased by 2 on a Y-o-Y basis, total 40| |
|People:|81,650 professionals as of March 31, 2024|
|Trailing 12 months attrition was 14.4%| |
---
# Deal Wins

|Leading financial services firm specializing in loans and mortgages|Selected LTIMindtree as primary engineering partner for regulatory timelines and seamless operations through 'Remediation-as-a-Service' and 'Operations-as-a-Service' engagement|
|---|---|
|Multinational financial services corporation|Awarded LTIMindtree a multi-year contract as exclusive assurance partner for transformation journey by implementing a centralized Quality Engineering organization|
|Global leader in insurance brokerage services and risk management solutions|Chose LTIMindtree as primary partner to establish a new technology platform in digital transformation journey|
|Leading energy company in the Middle East|Awarded LTIMindtree an end-to-end infrastructure-managed services contract for five years|
|Producer of frozen products in Sweden|Awarded LTIMindtree with a full-scope infrastructure contract for Cloud Transformation, a new logo for LTIMindtree|

# Partnerships

|Amazon Connect service delivery designation|Allows businesses to provide improved customer service at a lower cost th

In [18]:
from pathlib import Path

In [19]:
# document_path = Path("data/parsed_document.md")
document_path = Path ("parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

# Vector Embeddings

In [ ]:
!pip install unstructured

In [ ]:
!pip install "unstructured[all-docs]"

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [20]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.10/dist-packages/langdetect/detector_factory.py:88: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self.word_lang_prob_map[word] = [0.0] * langsize


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

11

In [22]:
print(docs[0].page_content)

LTIMindtree

Earnings Release & Fact Sheet

Fourth Quarter, Fiscal 2024

April 24, 2024

Contents

Press Release ................................................................. 3

Key Metrics ................................................................. 8

Financial Statements ....................................................... 11

Full Year Performance ..................................................... 14

Contact Information ......................................................... 19

Safe Harbour

Certain statements in this release concerning the future prospects are forward-looking statements. These statements by their nature involve risks and uncertainties that could cause the actual results to differ materially from such forward-looking statements. The Company assumes no obligation to revise or update any forward-looking statements that may be made from time to time by or on behalf of the Company.

Deal wins and recognitions section in this release include relevant 

In [23]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

In [24]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [25]:
%%time
query = "What is the most important innovation from LTIMindree?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 409 ms, sys: 0 ns, total: 409 ms
Wall time: 430 ms


In [26]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: About LTIMindtree

LTIMindtree is a global technology consulting and digital solutions company that enables enterprises across industries to reimagine business models, accelerate innovation, and maximize growth by harnessing digital technologies. As a digi

score: 0.7154869475877512
--------------------------------------------------------------------------------

text: Partnerships

Amazon Connect service delivery designation Allows businesses to provide improved customer service at a lower cost through cloud-based contact center service Joined the IBM Quantum Network To explore quantum computing innovation for global cl

score: 0.6452835251583553
--------------------------------------------------------------------------------

text: LTIMindtree

Earnings Release & Fact Sheet

Fourth Quarter, Fiscal 2024

April 24, 2024

Contents

Press Release ................................................................. 3

Key Metrics ........................................................

In [27]:

%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 246 ms, sys: 20 ms, total: 266 ms
Wall time: 266 ms


In [28]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: ac1f445eea99470faa5fd326416df2be

text: About LTIMindtree

LTIMindtree is a global technology consulting and digital solutions company that enables enterprises across industries to reimagine business models, accelerate innovation, and maximize growth by harnessing digital technologies. As a digi

--------------------------------------------------------------------------------

id: 01b5af84cfe643b485ef35fa090004e2

text: Partnerships

Amazon Connect service delivery designation Allows businesses to provide improved customer service at a lower cost through cloud-based contact center service Joined the IBM Quantum Network To explore quantum computing innovation for global cl

--------------------------------------------------------------------------------

id: 5ab255dd5f2c4c12abb91e25f9cc3b3a

text: LTIMindtree

Earnings Release & Fact Sheet

Fourth Quarter, Fiscal 2024

April 24, 2024

Contents

Press Release ................................................................. 3

Key Me

##Reranking

In [29]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 99.6MiB/s]


In [30]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.35 s, sys: 59.4 ms, total: 2.41 s
Wall time: 2.4 s


3

In [31]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: ac1f445eea99470faa5fd326416df2be

text: About LTIMindtree

LTIMindtree is a global technology consulting and digital solutions company that enables enterprises across industries to reimagine business models, accelerate innovation, and maximize growth by harnessing digital technologies. As a digi

score: 0.9769787788391113
--------------------------------------------------------------------------------

id: 01b5af84cfe643b485ef35fa090004e2

text: Partnerships

Amazon Connect service delivery designation Allows businesses to provide improved customer service at a lower cost through cloud-based contact center service Joined the IBM Quantum Network To explore quantum computing innovation for global cl

score: 0.6049167513847351
--------------------------------------------------------------------------------

id: 5ab255dd5f2c4c12abb91e25f9cc3b3a

text: LTIMindtree

Earnings Release & Fact Sheet

Fourth Quarter, Fiscal 2024

April 24, 2024

Contents

Press Release .......................

In [32]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [33]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [34]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [ ]:
%%time
response = qa.invoke("What is the most significant first quarter result of LTIMindtree?")

In [36]:
print_response(response)

Based on the provided information, there is no mention of the first quarter results of LTIMindtree.
The information provided is about the fourth quarter and full-year results ended March 31, 2024.
Therefore, I don't know the answer to this question.

However, I can provide some additional information about the company's performance. LTIMindtree
reported a revenue growth of 4.4% in USD terms for the full year ended March 31, 2024, with an EBIT
margin of 15.7%. The company also reported a strong order inflow of USD 5.6 billion, registering a
15.7% growth over FY23.


In [37]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [38]:
%%time
response = qa.invoke("What is the revenue for 2023 and % change?")

Running pairwise ranking..
CPU times: user 2.34 s, sys: 4.04 ms, total: 2.35 s
Wall time: 3.05 s


In [39]:
Markdown(response["result"])

**Answer:**

The revenue for 2023 (FY23) is $4,105.7 million (in USD) and ₹331,830 million (in INR).

The year-over-year (Y-o-Y) growth rate is 4.4% in USD and 7.0% in INR.

**Additional helpful information:**

* The revenue for 2024 (FY24) is $4,287.3 million (in USD) and ₹355,170 million (in INR), with a Y-o-Y growth rate of 4.4% in USD and 7.0% in INR.
* The revenue contribution by industry and geography is also provided in the given information.

In [40]:
%%time
response = qa.invoke("What is the revenue for 2022?")

Running pairwise ranking..
CPU times: user 2.21 s, sys: 6.58 ms, total: 2.22 s
Wall time: 2.63 s


In [41]:
print_response(response)

The revenue for 2022 (FY23) is $4,105.7 million in USD and ₹331,830 million in INR.

Additionally, the revenue growth rate from FY23 to FY24 is 4.4% in USD terms and 7.0% in INR terms.


In [42]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.25 s, sys: 7.48 ms, total: 2.26 s
Wall time: 3.39 s


In [43]:
print_response(response)

To calculate the revenue minus the costs and expenses for 2023, we need to find the Profit After Tax
(PAT) for FY23.

According to the financial statements, the PAT for FY23 is ₹44,103 million.

Here's a breakdown of the calculation:

Revenue: ₹331,830 million
Direct Cost: ₹232,037 million
Gross Profit: ₹99,793 million
SG&A Expenses: ₹38,716 million
EBITDA: ₹61,077 million
Depreciation and Amortization: ₹7,227 million
EBIT: ₹53,850 million
Forex Gains/(loss): ₹1,025 million
Other Income: ₹4,544 million
Finance Cost: ₹1,504 million
Provision for Tax: ₹13,812 million
PAT: ₹44,103 million

So, the revenue minus the costs and expenses for 2023 is ₹44,103 million, which is the PAT for FY23.

Additional helpful information:

* The EBITDA margin for FY23 is 18.4%.
* The EBIT margin for FY23 is 16.2%.
* The PAT margin for FY23 is 13.3%.
* The revenue growth rate for FY24 is 7.0% compared to FY23.


In [44]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2022? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.45 s, sys: 4.88 ms, total: 2.45 s
Wall time: 3.85 s


In [45]:
print_response(response)

Based on the provided financial statements, the revenue minus costs and expenses for FY23
(2022-2023) can be calculated as follows:

Revenue: ₹86,910 million (Q4 FY23)
Direct Cost: ₹60,923 million (Q4 FY23)
Gross Profit: ₹25,987 million (Q4 FY23)
SG&A Expenses: ₹9,950 million (Q4 FY23)
EBITDA: ₹16,037 million (Q4 FY23)
Depreciation and Amortization: ₹1,823 million (Q4 FY23)
EBIT: ₹14,214 million (Q4 FY23)
Forex Gains/(loss): ₹(523) million (Q4 FY23)
Other Income: ₹1,187 million (Q4 FY23)
Finance Cost: ₹436 million (Q4 FY23)
Provision for Tax: ₹3,301 million (Q4 FY23)
PAT: ₹11,141 million (Q4 FY23)

Revenue minus costs and expenses for FY23:
= Revenue - Direct Cost - SG&A Expenses - Depreciation and Amortization - Finance Cost
= ₹86,910 million - ₹60,923 million - ₹9,950 million - ₹1,823 million - ₹436 million
= ₹13,778 million

Additional helpful information:

* The company's EBITDA margin for FY23 is 18.4%, and the PAT margin is 13.3%.
* The company's revenue growth rate for FY23 is 2

In [46]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..
CPU times: user 3.28 s, sys: 1.76 ms, total: 3.28 s
Wall time: 4.17 s


In [47]:
Markdown(response["result"])

Unfortunately, the provided information does not contain the expected revenue for the second quarter of 2024. The financial highlights only provide information about the quarter ended March 31, 2024, and the year ended March 31, 2024, but not about future quarters.

However, I can provide some additional information that might be helpful:

* The revenue for the quarter ended March 31, 2024, was $1,069.4 million, with a growth of 1.1% year-over-year.
* The revenue for the year ended March 31, 2024, was $4,287.3 million, with a growth of 4.4% year-over-year.
* The company has a strong order inflow, with a growth of 15.7% over FY23, totaling $5.6 billion.

Please note that these figures are historical and do not provide any information about future quarters.

In [50]:
%%time
response = qa.invoke("What is the overall outlook of Q4 2024?")

Running pairwise ranking..
CPU times: user 2.24 s, sys: 1.88 ms, total: 2.25 s
Wall time: 3.08 s


In [51]:
print_response(response)

**Overall Outlook of Q4 2024:**

The overall outlook of Q4 2024 is negative, with a decline in revenue and net profit compared to the
previous quarter. Revenue decreased by 1.3% Q-o-Q, and net profit decreased by 5.8% Q-o-Q.

**Additional Helpful Information:**

* Despite the decline in Q4, the company reported a growth of 4.4% in USD terms for the full year
ended March 31, 2024.
* The company's order inflow for the full year was USD 5.6 billion, registering a 15.7% growth over
FY23.
* The company's CEO, Debashis Chatterjee, mentioned that they closed FY24 amidst a tough macro
environment and delivered a resilient performance.
